In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 

2025-04-24 14:48:28.120900: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-24 14:48:28.124165: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-24 14:48:28.132361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745480908.146628   52152 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745480908.150609   52152 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745480908.162704   52152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'INTC', 'AMD']

### Download 2 Years of Stock Data

In [41]:
data = {}

for ticker in tickers:
    df = yf.download(ticker, period="2y")
    df.dropna(inplace=True)
    data[ticker] = df[['Open', 'High', 'Low', 'Close', 'Volume']]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Save Data to .csv

In [42]:
# Save the data in df to CSV
for ticker, df in data.items():
    df.to_csv(f'Stocks/{ticker}_data.csv')

Load Data from .csv

In [4]:
# Load the data from CSV
data = {}
for ticker in tickers:
    df = pd.read_csv(f'Stocks/{ticker}_data.csv', index_col=0, parse_dates=True)
    # Convert columns to numeric, coercing errors to NaN, and drop rows with NaN values
    df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].apply(pd.to_numeric, errors='coerce')
    df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)
    data[ticker] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype({'Open': 'float64', 'High': 'float64', 'Low': 'float64', 'Close': 'float64', 'Volume': 'int64'})

/tmp/ipykernel_52152/4098107145.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f'Stocks/{ticker}_data.csv', index_col=0, parse_dates=True)
/tmp/ipykernel_52152/4098107145.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f'Stocks/{ticker}_data.csv', index_col=0, parse_dates=True)
/tmp/ipykernel_52152/4098107145.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f'Stocks/{ticker}_data.csv', index_col=0, parse_dates=True)
/tmp/ipykernel_52152/4098107145.py:4: UserWarning: Could not infer format, so each element will b

### Build Dataset (Sequences + Labels)

In [5]:
WINDOW = 60
PREDICT_DAYS = 30

X, y = [], []

for ticker, df in data.items():
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    
    for i in range(len(scaled) - WINDOW - PREDICT_DAYS):
        X_seq = scaled[i:i+WINDOW]
        close_start = df.iloc[i+WINDOW-1]['Close']
        close_end = df.iloc[i+WINDOW+PREDICT_DAYS-1]['Close']
        growth = (close_end - close_start) / close_start
        X.append(X_seq)
        y.append(growth)

X = np.array(X)
y = np.array(y).reshape(-1, 1)


### Split Dataset

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)

### Build the LSTM Model


In [9]:
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

2025-04-24 14:51:15.732801: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/kamin/Documents/CPE 232 Project/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,985 (70.25 KB)

 Trainable params: 17,985 (70.25 KB)

 Non-trainable params: 0 (0.00 B)

### Train the Model

In [10]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=64
)

Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0189 - val_loss: 0.0194
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0188 - val_loss: 0.0187
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0179 - val_loss: 0.0191
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0182 - val_loss: 0.0186
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0177 - val_loss: 0.0188
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0167 - val_loss: 0.0192
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0173 - val_loss: 0.0184
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0180 - val_loss: 0.0184
Epoch 9/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0174 - val_loss: 0.0183
Epoch 10/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0181 - val_loss: 0.0184


### Predict Growth on Latest Data & Pick Top Stock

In [48]:
def predict_next_growth(df, window=60):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df[-window:])
    X_input = np.expand_dims(scaled, axis=0)
    pred = model.predict(X_input, verbose=0)
    return pred[0][0]

predictions = {ticker: predict_next_growth(df) for ticker, df in data.items()}
top_stock = max(predictions, key=predictions.get)

print("Predicted 30-day growth:")
for t, p in predictions.items():
    print(f"{t}: {p:.4f}")

print(f"\n📈 Best stock to pick: {top_stock}")


Predicted 30-day growth:
AAPL: 0.0276
GOOGL: 0.0488
MSFT: 0.0546
AMZN: 0.0827
META: 0.0929
TSLA: -0.0443
NFLX: 0.0273
NVDA: 0.0388
INTC: 0.0539
AMD: 0.0360

📈 Best stock to pick: META


### Prepare Data for All 10 Stocks in Parallel

In [49]:
WINDOW = 60
PREDICT_DAYS = 30
NUM_STOCKS = len(tickers)

# Ensure all stocks have the same length
min_len = min(len(df) for df in data.values())
for k in data:
    data[k] = data[k].iloc[-min_len:]

# Stack all stocks into a 3D array: (samples, 10, 60, 5)
X_multi, y_multi = [], []

for i in range(min_len - WINDOW - PREDICT_DAYS):
    X_seq = []
    y_seq = []
    for ticker in tickers:
        df = data[ticker]
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(df.iloc[i:i+WINDOW])
        X_seq.append(scaled)

        # Label: growth over next 30 days
        close_start = df.iloc[i+WINDOW-1]['Close']
        close_end = df.iloc[i+WINDOW+PREDICT_DAYS-1]['Close']
        growth = (close_end - close_start) / close_start
        y_seq.append(growth)

    X_multi.append(X_seq)
    y_multi.append(y_seq)

X_multi = np.array(X_multi)  # shape: (samples, 10, 60, 5)
y_multi = np.array(y_multi)  # shape: (samples, 10)


### Split the Dataset

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X_multi, y_multi, test_size=0.2, shuffle=True)

###  Build the Multi-stock, Multi-output LSTM Model

In [51]:
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(NUM_STOCKS, WINDOW, 5))  # (batch, 10, 60, 5)

# LSTM per stock via TimeDistributed
x = TimeDistributed(LSTM(64))(input_layer)  # (batch, 10, 64)
output_layer = Dense(1)(x)  # (batch, 10, 1)
output_layer = tf.keras.layers.Lambda(lambda t: tf.squeeze(t, axis=-1))(output_layer)  # (batch, 10)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 10, 60, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 10, 64)         │        17,920 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10, 1)          │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_1 (Lambda)               │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,985 (70.25 KB)

 Trainable params: 17,985 (70.25 KB)

 Non-trainable params: 0 (0.00 B)

### Train the Model

In [52]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 139ms/step - loss: 0.0268 - val_loss: 0.0198
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0205 - val_loss: 0.0186
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0210 - val_loss: 0.0181
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0200 - val_loss: 0.0180
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0188 - val_loss: 0.0181
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0203 - val_loss: 0.0181
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0199 - val_loss: 0.0182
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0205 - val_loss: 0.0181
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0200 - val_loss: 0.0178
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0191 - val_loss: 0.0180


### Predict Best Stock from Latest Data

In [53]:
# Prepare last 60 days for each stock
X_latest = []

for ticker in tickers:
    df = data[ticker]
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df[-WINDOW:])
    X_latest.append(scaled)

X_latest = np.expand_dims(X_latest, axis=0)  # shape: (1, 10, 60, 5)

# Predict growth for all 10 stocks
growths = model.predict(X_latest, verbose=0)[0]
growth_dict = dict(zip(tickers, growths))

top_stock = max(growth_dict, key=growth_dict.get)

print("Predicted 30-day growth per stock:")
for t, g in growth_dict.items():
    print(f"{t}: {g:.4f}")

print(f"\n📈 Best stock to pick: {top_stock}")


Predicted 30-day growth per stock:
AAPL: -0.0045
GOOGL: -0.0141
MSFT: -0.0126
AMZN: -0.0045
META: -0.0103
TSLA: -0.0379
NFLX: -0.0067
NVDA: -0.0102
INTC: 0.0125
AMD: 0.0025

📈 Best stock to pick: INTC


### Evaluate Model Performance

In [ ]:
# Evaluate on validation set
val_loss = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation MSE: {val_loss:.6f}")

# Make predictions on validation set
y_pred = model.predict(X_val)

# Calculate RMSE
rmse = np.sqrt(np.mean((y_pred - y_val) ** 2))
print(f"RMSE: {rmse:.6f}")

# Calculate Mean Absolute Error
mae = np.mean(np.abs(y_pred - y_val))
print(f"MAE: {mae:.6f}")    

Validation MSE: 0.018014
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step 
RMSE: 0.134217
MAE: 0.097405
